In [ ]:
import cv2
import numpy as np

def find_and_draw_object(object_path, scene_path):
    # Read the images
    object_img = cv2.imread(object_path)
    scene_img = cv2.imread(scene_path)
    
    # Convert images to grayscale
    object_gray = cv2.cvtColor(object_img, cv2.COLOR_BGR2GRAY)
    scene_gray = cv2.cvtColor(scene_img, cv2.COLOR_BGR2GRAY)

    # Initialize SIFT detector
    sift = cv2.SIFT_create()

    # Find keypoints and descriptors
    keypoints1, descriptors1 = sift.detectAndCompute(object_gray, None)
    keypoints2, descriptors2 = sift.detectAndCompute(scene_gray, None)

    # Initialize FLANN matcher
    FLANN_INDEX_KDTREE = 1
    index_params = dict(algorithm=FLANN_INDEX_KDTREE, trees=5)
    search_params = dict(checks=50)
    flann = cv2.FlannBasedMatcher(index_params, search_params)

    # Match descriptors
    matches = flann.knnMatch(descriptors1, descriptors2, k=2)

    # Apply Lowe's ratio test to filter good matches
    good_matches = []
    for m, n in matches:
        if m.distance < 0.7 * n.distance:
            good_matches.append(m)

    # Extract location of good matches
    if len(good_matches) >= 4:
        src_pts = np.float32([keypoints1[m.queryIdx].pt for m in good_matches]).reshape(-1, 1, 2)
        dst_pts = np.float32([keypoints2[m.trainIdx].pt for m in good_matches]).reshape(-1, 1, 2)

        # Find homography
        H, mask = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC, 5.0)
        
        # Get dimensions of object image
        h, w = object_gray.shape
        
        # Define corners of object image
        pts = np.float32([[0, 0], [0, h-1], [w-1, h-1], [w-1, 0]]).reshape(-1, 1, 2)
        
        # Transform corners to scene coordinates
        dst = cv2.perspectiveTransform(pts, H)
        
        # Draw border around detected object
        result = cv2.polylines(scene_img, [np.int32(dst)], True, (0, 255, 0), 3, cv2.LINE_AA)
        
        # Draw matches (optional)
        match_img = cv2.drawMatches(object_img, keypoints1, scene_img, keypoints2, good_matches, None,
                                  flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)
        
        # Display results
        cv2.imshow('Matches', match_img)
        cv2.imshow('Result', result)
        cv2.waitKey(0)
        cv2.destroyAllWindows()
        
    else:
        print("Not enough good matches found - %d/%d" % (len(good_matches), 4))

In [ ]:
object_path = 'Object.jpg'  # Path to your object image
scene_path = 'Picture.jpg'    # Path to your scene image
find_and_draw_object(object_path, scene_path)